In [ ]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
from pytesseract import image_to_string

In [ ]:
patient_medic_events = pd.read_csv("patient_medicine_events_at_QA_by_label.txt", sep='\t')

In [ ]:
floors = ['A', 'B', 'C', 'D', 'E', 'F', 'G']

In [ ]:
for floor in floors:
    plt.figure()
    img=mpimg.imread('level_maps_'+floor.lower()+'.png')
    plt.figure(figsize=(60,6))
    imgplot = plt.imshow(img)
    fig = plt.gca()
    print(image_to_string(Image.open(img)))